In [166]:
import stanfordnlp
stanfordnlp.download('en')
nlp = stanfordnlp.Pipeline(processors = "tokenize,mwt,lemma,pos")
doc = nlp("Event Management is the process that monitors all events that occur through the IT infrastructure to allow for normal operation and also to detect and escalate exception conditions.")
doc.sentences[0].print_tokens()


Using the default treebank "en_ewt" for language "en".
Would you like to download the models for: en_ewt now? (Y/n)

Default download directory: C:\Users\User\stanfordnlp_resources
Hit enter to continue or type an alternate directory.

Download location: C:\Users\User\stanfordnlp_resources\en_ewt_models.zip

Download complete.  Models saved to: C:\Users\User\stanfordnlp_resources\en_ewt_models.zip
Extracting models file for: en_ewt
Cleaning up...Done.
Use device: cpu
---
Loading: tokenize
With settings: 
{'model_path': 'C:\\Users\\User\\stanfordnlp_resources\\en_ewt_models\\en_ewt_tokenizer.pt', 'lang': 'en', 'shorthand': 'en_ewt', 'mode': 'predict'}
---
Loading: lemma
With settings: 
{'model_path': 'C:\\Users\\User\\stanfordnlp_resources\\en_ewt_models\\en_ewt_lemmatizer.pt', 'lang': 'en', 'shorthand': 'en_ewt', 'mode': 'predict'}
Building an attentional Seq2Seq model...
Using a Bi-LSTM encoder
Using soft attention for LSTM.
Finetune all embeddings.
[Running seq2seq lemmatizer with ed

100%|██████████| 235M/235M [04:02<00:00, 873kB/s] 


In [182]:
print(*[f'text: {word.text+" "}\tlemma: {word.lemma}\tupos: {word.upos}\txpos: {word.xpos}' for sent in doc.sentences for word in sent.words], sep='\n')




text: Event         	lemma: event	upos: NOUN	xpos: NN
text: Management         	lemma: management	upos: NOUN	xpos: NN
text: is         	lemma: be	upos: AUX	xpos: VBZ
text: the         	lemma: the	upos: DET	xpos: DT
text: process         	lemma: process	upos: NOUN	xpos: NN
text: that         	lemma: that	upos: PRON	xpos: WDT
text: monitors         	lemma: monitor	upos: VERB	xpos: VBZ
text: all         	lemma: all	upos: DET	xpos: DT
text: events         	lemma: event	upos: NOUN	xpos: NNS
text: that         	lemma: that	upos: PRON	xpos: WDT
text: occur         	lemma: occur	upos: VERB	xpos: VBP
text: through         	lemma: through	upos: ADP	xpos: IN
text: the         	lemma: the	upos: DET	xpos: DT
text: IT         	lemma: it	upos: NOUN	xpos: NN
text: infrastructure         	lemma: infrastructure	upos: NOUN	xpos: NN
text: to         	lemma: to	upos: PART	xpos: TO
text: allow         	lemma: allow	upos: VERB	xpos: VB
text: for         	lemma: for	upos: ADP	xpos: IN
text: normal         	le

In [13]:
with open("ITIL Books/ITIL 3/Service operation glossary/definitions.txt") as file:
    glossary_definitions = file.read()

definitions = [i.lower().replace("\n", " ").split('::') for i in glossary_definitions.split('\n\n')]

concepts_from_definitions = [i[0].strip() for i in definitions]

with open("ITIL Books/ITIL 3/Service operation chapter 4/Service operation chapter 4 - all.txt") as file:
    text = file.read()

ontology = {}

for index in range(0, len(definitions)):
    ontology[definitions[index][0].strip()] = [f"definition::{definitions[index][1].strip()}"]

import nltk
# Tokenize into sentences
sentences = nltk.sent_tokenize(text)

# Split sentences into words
sentences = [nltk.word_tokenize(i.lower().replace("\n", " ")) for i in sentences]

sentences_containing_concepts = {}
for sentence in sentences:
    # print("---")
    count = 0
    for concept in concepts_from_definitions:
        concept_list = concept.split()
        
        if concept_list[0] in sentence:
            add_to_sentences_containing_concepts = True
            
            if len(concept_list) > 1:
                current_word_index = sentence.index(concept_list[0])
                concept_word_length_count = 1
                for concept_word in concept_list[1::]:
                    current_word_index = current_word_index + 1
                    
                    if sentence[current_word_index] != concept_word:
                        add_to_sentences_containing_concepts = False
                        break
                
            if add_to_sentences_containing_concepts is True:
                sentence_string = " ".join(sentence)
                if sentence_string not in sentences_containing_concepts.keys():
                    sentences_containing_concepts[sentence_string] = []
                sentences_containing_concepts[sentence_string].append(concept)
            

# sentences_containing_concepts = [i for i in sentences if i in concepts_from_definitions]
 
# for sentence in sentences_containing_concepts:
#     print(sentence)

# print(sentences_containing_concepts)

# print(ontology)

for sentence, concepts in sentences_containing_concepts.items():
    for concept in concepts:
        ontology[concept].append(sentence)
        
# TODO: In definitions replace (book name) with own in book relation


In [14]:
import re
import random

def load_templates(file_path):
    with open(file_path) as template_file:
        templates = template_file.readlines()
    return templates

# question_template = "question_template(Question_number=1,Answer_number=4)::Question->What is the definition for {concept.name}?::Answer->{concept.definition}"
templates = load_templates("templates/question_templates.txt")
# question_template = templates[random.randint(0, len(templates) - 1)]
question_template = templates[0]

# print(templates)
# print(question_template)

# nltk.tokenize.mwe 
        
def get_definition(concept_name):
    return [i.split("::")[1] for i in ontology[concept_name] if i.split("::")[0] == "definition"][0]

class Concept:
    def __init__(self, name, ontology):
        self.name = name
        self.ontology = ontology
        self.attributes = ontology[self.name]
        self.definition = get_definition(self.name)
        self.book = re.findall("^\w*\(.*?\)", self.definition) 
        # TODO: Fix book

class Question:
    def __init__(self, question_string, answer_list, question_params):
        self.question_string = question_string
        self.answer_list = answer_list
        self.parameters = question_params
        
    def randomise_answers(self):
        correct_answer = self.answer_list[0]
        random.shuffle(self.answer_list)
        return self.answer_list.index(correct_answer)
    
    def print_question_and_answer(self):
        print(self.question_string)
        self.randomise_answers()
        for answer in self.answer_list:
            print(f"- " + answer.strip("\n"))
        
    def format_output(self):
        self.print_question_and_answer()
        
def get_incorrect_answers(correct_concept, question_parameters):
    # TODO: Move literal strings into a central Constants file so they can be easily changed
    number_of_answers = question_parameters["Answer_number"] if "Answer_number" in question_parameters.keys() else 4
    answer_type = question_parameters["Answer_type"] if "Answer_type" in question_parameters.keys() else None
    
    incorrect_answers = []
    
    while(len(incorrect_answers) < int(number_of_answers) - 1):
        incorrect_concept_name = list(ontology.keys())[random.randint(0, len(ontology))]
        if incorrect_concept_name != correct_concept.name and answer_type == "definition":
            incorrect_answers.append(get_definition(incorrect_concept_name))
        elif incorrect_concept_name != correct_concept.name and answer_type == "concept.name":
            incorrect_answers.append(incorrect_concept_name)
    # TODO: Make answer picker more generalised
    
    return incorrect_answers
    
# def get_random_attribute_without_concept_name(concept):
#     print(concept.attributes[0].split("::")[])    
    
def replace_template_variables(template_format_string, concept):
    replaced_string = template_format_string
    
    # print(replaced_string)
    
    # get_random_attribute_without_concept_name(concept)
    
    replaced_string = replaced_string.replace("{!concept.name}", "_" * 10)
    replaced_string = replaced_string.replace("{newline}", "\n")
    replaced_string = replaced_string.replace("{concept.name}", concept.name)
    replaced_string = replaced_string.replace("{concept.definition}", concept.definition)
    
    # replaced_string = re.sub("{.*?\!concept\.name.*?}", "__________", replaced_string)
    # replaced_string = re.sub("{.*?newline}.*?", "\n", replaced_string)
    # replaced_string = re.sub("{.*?concept\.name.*?}", concept.name, replaced_string)
    # replaced_string = re.sub("{.*?concept\.definition.*?}", concept.definition, replaced_string)\
    # replaced_string = re.sub("{.*?concept\.relation\.attribute*?}", concept.attrubute, replaced_string)
    # replaced_string = re.sub("{.*?concept\.book.*?}", concept.book, replaced_string)
    
    # print(replaced_string)
    
    return replaced_string

def parse_template_question(question_template_to_parse, concept_name=""):
    template_list = question_template_to_parse.split("::")
    question_parameters = template_list[0].split("(")[1].replace(")", "").split(",")    
    question_parameters = {item.split("=")[0]: item.split("=")[1] for item in question_parameters}
    question = template_list[1].split("->")[1]
    answer = template_list[2].split("->")[1]
        
    if concept_name == "":
        concept_name = list(ontology.keys())[random.randint(0, len(ontology))]     
        
    concept = Concept(concept_name, ontology)
    
    # print(concept.definition)
    # print(concept.book)
    
    question = replace_template_variables(question, concept)
    answer = replace_template_variables(answer, concept)
    
    answer_list = [answer]
                
    answer_list = answer_list + get_incorrect_answers(concept, question_parameters)
                
    return Question(question, answer_list, question_parameters)

def parse_template(template_to_parse, concept_name=""):
    if "question_template" in template_to_parse.split("::")[0]:
        return parse_template_question(template_to_parse)
    else:
        print(f"Error parsing template {template_to_parse}")
        return None
 
parsed_template = parse_template(question_template)

parsed_template.format_output()

# for sentence in sentences:
#     print(nltk.word_tokenize(sentence))
    
# print(nltk.tokenize.TextTilingTokenizer().tokenize(glossary_definitions))
# print(nltk.word_tokenize(glossary_definitions))


What is the definition for "quality"?
-  (service operation) a record containing the details of a problem. each problem record documents the lifecycle of a single problem.
-  the ability of a product, service, or process to provide the intended value. for example, a hardware component can be considered to be of high quality if it performs as expected and delivers the required reliability. process quality also requires an ability to monitor effectiveness and efficiency, and to improve them if necessary. see also quality management system.
-  (service operation) monitoring that looks for patterns of events to predict possible future failures. see also reactive monitoring.
-  information in readable form. a document may be paper or electronic. for example, a policy statement, service level agreement, incident record, diagram of computer room layout. see also record.


In [15]:
for concept in ontology:
    print(concept)
    for relationship in ontology[concept]:
        print(f"\t-{relationship}")



acceptance
	-definition:: formal agreement that an it service, process, plan, or other deliverable is complete, accurate, reliable and meets its specified requirements. acceptance is usually preceded by evaluation or testing and is often required before proceeding to the next stage of a project or process.
	-other problem records , and corresponding known error records , may be triggered in testing , particularly the latter stages of testing such as user acceptance testing/trials ( uat ) , if a decision is made to go ahead with a release even though some faults are known .
access management
	-definition:: (service operation) the process responsible for allowing users to make use of it services, data, or other assets. access management helps to protect the confidentiality, integrity and availability of assets by ensuring that only authorized users are able to access or modify the assets. access management is sometimes referred to as rights management or identity management.
	-ï¿½ access

KeyboardInterrupt: 

In [43]:
# # Code from https://spacy.io/
# import spacy
# 
# nlp = spacy.load("en_core_web_sm")
# 
# text = "Event Management is the process that monitors all events that occur through the IT infrastructure to allow for normal operation and also to detect and escalate exception conditions."
# 
# doc = nlp(text)
# 
# # # Analyze syntax
# print("Noun phrases:", [chunk.text for chunk in doc.noun_chunks])
# print("Verbs:", [token.lemma_ for token in doc if token.pos_ == "VERB"])
# 
# print("-" * 100)
# 
# # Find named entities, phrases and concepts
# for entity in doc.ents:
#     print(entity.text, entity.label_)
# 
# print("-" * 100)
# for token in doc:
#     print(token.text, token.pos_, token.dep_)
# 
# print("-" * 100)
# for token in doc:
#     print(token.text, token.dep_)

# from spacy import displacy
# displacy.render(doc, style="dep", jupyter=True, options={'distance': 140})



Noun phrases: ['Event Management', 'the process', 'all events', 'the IT infrastructure', 'normal operation', 'exception conditions']
Verbs: ['be', 'monitor', 'occur', 'allow', 'detect', 'escalate']
----------------------------------------------------------------------------------------------------
Event Management ORG
----------------------------------------------------------------------------------------------------
Event PROPN compound
Management PROPN nsubj
is VERB ROOT
the DET det
process NOUN attr
that DET nsubj
monitors VERB relcl
all DET det
events NOUN dobj
that DET nsubj
occur VERB relcl
through ADP prep
the DET det
IT NOUN compound
infrastructure NOUN pobj
to PART aux
allow VERB advcl
for ADP prep
normal ADJ amod
operation NOUN pobj
and CCONJ cc
also ADV advmod
to PART aux
detect VERB conj
and CCONJ cc
escalate VERB conj
exception NOUN compound
conditions NOUN dobj
. PUNCT punct
--------------------------------------------------------------------------------------------------